# Thinking in tensors, writing in PyTorch

A hands-on course by [Piotr Migdał](https://p.migdal.pl) (2019).
This notebook prepared by [Weronika Ormaniec](https://github.com/werkaaa) and Piotr Migdał.

## Notebook 3: Linear regression

<a href="https://colab.research.google.com/github/stared/thinking-in-tensors-writing-in-pytorch/blob/master/3%20Linear%20regression.ipynb"  target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg"/>
</a>


[Linear regression](https://en.wikipedia.org/wiki/Linear_regression) is one of the most common predictive models. In plain words, we fit a straight line that fits to the data. Mathematically speaking, we use linear combination of input variables to predict the output variable.

$$y = a_1 x_1 + \ldots + a_n x_n + b$$

Before moving any further, try to experience it viscerally with [Ordinary Least Squares Regression
Explained Visually - Visually Explained](http://setosa.io/ev/ordinary-least-squares-regression/):

![http://setosa.io/ev/ordinary-least-squares-regression/](imgs/linreg_setosa.png)

However, it occurs that lots of dependencies in the actual world can be described just by fitting a linear equation to the observed data. That's what we are going to do now!

In Python we typically use [LinearRegression from scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). Here we use PyTorch to show everything step-by-step. Moreover, linear regression is a building block of any regression with deep learning - so it is good to understand it well!

In [ ]:
!pip install --quiet livelossplot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn

from livelossplot import PlotLosses
from ipywidgets import interact, fixed

### Data

Have you ever wondered what is the relation between brain and body weights among various animal species?

Let's try a [Brain to Body Weight Dataset](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_Brain2BodyWeight)!

> These data represent the relation between weights of the body and brain of various species. It may be used to discuss bivariate exploratory and quantitative data analyses in the case of allometric relationships. Brain-to-body weight ratio is assumed to be related to species intelligence. The encephalization quotient is a more complex measurement that takes into account allometric effects of widely divergent body sizes across several taxa. The brain-to-body mass ratio is a simpler measure of encephalization within species.

In [ ]:
data = pd.read_csv("data/Animals.csv", index_col='Species')
data

In [ ]:
# or sorted in a different way
data.sort_values(by="BrainWeight(kg)", ascending=False).head(10)

So, which is the smartest one? 

If we go by brain to body weight proportion, humans are on the top, but so are hamsters (one can argue that there are smarter creatures on the list).

If we go by brain weight, it favors bigger animals. Sure, whales and elephants are smart - but are they THAT smarter than humans?

In [ ]:
# Let's make a scatter plot
data.plot.scatter(x="BodyWeight(kg)", y="BrainWeight(kg)")

At first glance it does not resemble any particular dependance. However, if we change the scale something interesting can be spotted with [logarithmic scaling](https://simple.wikipedia.org/wiki/Logarithmic_scale):

In [ ]:
data.plot.scatter(x="BodyWeight(kg)", y="BrainWeight(kg)", logx=True, logy=True);

We see a clear dependence that the bigger body weight (on average) the bigger brain weight.
Let's investigate that!

First of all, we need to prepare the data. We see some dependence on the scatter plot only after scaling both x and y axes logarithmically. That is why, if we want to see the same relationship in the data itself, we take natural logarithm of brain and body weights.

In [ ]:
X = np.log(data['BodyWeight(kg)'])
Y = np.log(data['BrainWeight(kg)'])

### A toy example


At the beginning let's take a look only on a few points from the dataset.

In [ ]:
X_less = X[::6]
X_less

In [ ]:
Y_less = Y[::6]
Y_less

In [ ]:
fig, ax = plt.subplots(figsize=(5, 6))
ax.scatter(X_less, Y_less, color='r')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_ylim([-6, 4]);

On the scatterplot it can be seen that the relationship between presented data is almost linear. We will try to apply the equation:

$$ y = ax+b$$

to the analysed dataset. The only problem is how to find $a$ and $b$.
Try to find a proper line manually!

In [ ]:
def plot_model(a, b, X, Y):
    X = np.sort(X)
    Y_pred = a * X + b
    fig, ax = plt.subplots(figsize=(5, 6))
    ax.plot(X, Y_pred)
    ax.scatter(X, Y, color='r')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_ylim([-6, 4])
    
interact(plot_model, 
         a=(-4.0, 4.0), 
         b=(-6.0, 6.0),
         X=fixed(X_less),
         Y=fixed(Y_less)
        );

### Loss function

We will try to somehow measure if the coefficients in the equation are good enough to describe our problem. In order to do it we will define a loss function - an equation that will tell us how much our approximation differs from the expected output. 

The loss function should:

* depend only on the coefficients of the model, expected output and our approximation,
* shrink if our approximation is becoming better and grow if it gets worse.

When it comes to linear regression the most common approach is the [least-squares loss function](https://en.wikipedia.org/wiki/Least_squares). We will calculate the average square of the vertical deviations from each data point to the line. Since we first square the deviations, it does not matter if the data point is above or below the line. 


$$y^{ pred}_{i} = ax_{i}+b $$
$$L=\frac{1}{N}\sum_{i=0}^{N-1}( y^{pred}_{i} - y_{i})^2 $$

In [ ]:
aa_ = np.linspace(-2, 3, num=100)
bb_ = np.linspace(-8, 8, num=100)
aa, bb = np.meshgrid(aa_, bb_)

def loss_numpy(aa, bb, X, Y):
    loss = np.zeros_like(aa)
    for i in range(len(loss)):
        for j in range(len(loss[0])):
            loss[i][j] = ((aa[i,j] * X + bb[i,j] - Y)**2).sum()
    return loss

cs = plt.contour(aa, bb, np.sqrt(loss_numpy(aa, bb, X_less, Y_less)), cmap='coolwarm')
plt.clabel(cs, inline=1, fontsize=10)
plt.title("Loss")
plt.xlabel('a')
plt.ylabel('b')

If we want to use PyTorch in our model we need tensors!

In [ ]:
X_less = torch.tensor(X_less)
X_less

In [ ]:
Y_less = torch.tensor(Y_less)
Y_less

In [ ]:
def Y_pred(A, B, X):
    return A * X + B

In [ ]:
def loss(Y_pred, Y):
    return (Y_pred - Y).pow(2).mean()

### Minimizing loss function

Since we have defined the loss function, we should minimize it. 
There is an explicit formula for the coefficients that will guarantee the best fit of the line for particular data:

$$a=\frac{\sum_{i=0}^{N-1}{x_i} \cdot \sum_{i=0}^{N-1}{y_i}  - \sum_{i=0}^{N-1}{x_iy_i} }{(\sum_{i=0}^{N-1}{x_i})^2-\sum_{i=0}^{N-1}{x_i^2} }$$
$$b=\frac{1}{N}\left(\sum_{i=0}^{N-1}{y_i} -a\sum_{i=0}^{N-1}{x_i}\right) $$

You can see also [expected value](https://en.wikipedia.org/wiki/Expected_value) notation, were $\mathbb{E}[x]$ means an average of $x$, that is $\sum_{i=0}^{N-1}{x_i}/N$.

$$a = \frac{\mathbb{E}[x]\mathbb{E}[y] - \mathbb{E}[xy]}{(\mathbb{E}[x])^2 - \mathbb{E}[x^2]}$$
$$b = \mathbb{E}[y] - a E[x] $$

However, for didactic purpose we will minimize the loss function differently, using **gradient descent** (discussed in the previous notebook).

By doing so we will step by step rotate and move the line, so it will reflect the actual location of data points. In order to do it we need to repeatedly shift the weights till we find a minimum of the loss function. What we need is a mathematical operation that will tell us how the loss function change, if we increase or decrease $a$ and $b$. The operation we are looking for is partial derivative:

$$\dfrac{\partial L}{\partial a}  = \frac{2}{N}\sum_{i=0}^{N-1} (y^{pred}_{i} -y_{i}) \cdot x_{i}$$ 

$$\dfrac{\partial L}{\partial b} = \frac{2}{N}\sum_{i=0}^{N-1} (y^{pred}_{i} -y_{i})$$

Let's write them as code:

In [ ]:
def dL_da(A, B, X, Y):
    Y_prediction = Y_pred(A, B, X)
    return 2 * ((Y_prediction - Y) * X).mean()

In [ ]:
def dL_db(A, B, X, Y):
    Y_prediction = Y_pred(A, B, X)
    return 2 * (Y_prediction - Y).mean()

We need to specify two more things:

- **learning\_rate** - hyperparameter that will define how much the value of the derivative will influence the change of $a$ and $b$,
- **num\_epochs** - hyperparameter defining how many iterations it will take to sufficiently minimize the loss function.

In [ ]:
def train_model_manually(X, Y, A, B, learning_rate, num_epochs):
    logs = {}
    epoch_loss = 0.0
    
    def extra_plot(*args):
        plt.plot(X.numpy(), Y.numpy(), 'r.', label="Ground truth")
        plt.plot(X.numpy(), Y_pred(A, B, X).numpy(), '-', label="Model")
        plt.title("Prediction")
        plt.legend(loc='lower right')
        
    liveloss = PlotLosses(extra_plots=[extra_plot], plot_extrema=False)
    
    for i in range(num_epochs):
        A -= learning_rate * dL_da(A, B, X, Y)
        B -= learning_rate * dL_db(A, B, X, Y)
        
        epoch_loss = loss(Y_pred(A, B, X), Y)
        
        avg_loss = epoch_loss / len(X)
        

        liveloss.update({
            'loss': avg_loss,
        })
        liveloss.draw()
        
        print("y = {:.3f}x{:+.3f}".format(A.item(), B.item()))

Last thing we need to do before trying to minimalize the loss function is to initialize the coefficients with some random values.

In [ ]:
A = torch.randn(1)
A

In [ ]:
B = torch.randn(1)
B

In [ ]:
train_model_manually(X_less, Y_less, A, B, learning_rate=0.025, num_epochs=100)

And that is how we find the propper line!

### Linear regression using PyTorch

Knowing how linear regression works, let's come back to the relation between body and brain weights. This time we will use built-in PyTorch functions.

Firstly, we need to prepare the data. PyTorch built-in models have specified shapes of the input data. It is all specified in [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#linear). Below, we are changing the shape of our data into format **(number_of_datapoints, in_features)**.

In [ ]:
X = torch.tensor(X).view(-1, 1)
X.size()

In [ ]:
Y = torch.tensor(Y).view(-1, 1)
Y.size()

Instead of initializing the coefficients manually, we can define the model using a built in class. Since both input and output in the analyzed problem have only one dimension we set **(in_features=1, out_features=1)** as arguments of **nn.Linear**. That is also specified in [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#linear).

In [ ]:
linear_model = nn.Linear(in_features=1, out_features=1)
print(linear_model.weight)
print(linear_model.bias)

Instead of **gradient\_step** function, we will define an **optimizer** with learning rate and built-in **loss function**.

In [ ]:
optim = torch.optim.SGD(linear_model.parameters(), lr=0.03)
loss_function = nn.MSELoss()
loss = loss_function(linear_model(X), Y)
print(loss)  

Before training the model, let's see what does the line with random coefficients look like.

In [ ]:
default_animals_to_print = {'Elephant', 'Adult_Human', 'Alligator',
                            'Owl', 'Cat', 'Chimpanzee',
                            'Green_Lizard', 'Hamster', 'Cow'}

def plot_model_annotate(X, Y, A, B, labels,
                        animals_to_print=default_animals_to_print):
    fig, ax = plt.subplots()
    ax.scatter(X, Y, color='red')
    y_pred = A * X + B
    ax.plot(X, y_pred)
    ax.set_xlabel("LogBodyWeight(kg)")
    ax.set_ylabel("LogBrainWeight(kg)")
    ax.set_ylim([-8, 4])
    
    for i, label in enumerate(labels):
        if animals_to_print is None or label in animals_to_print: 
            ax.annotate(label, (X[i], Y[i]))
   
    print("LogBodyWeight(kg) = {:.3f}*LogBrainWeight(kg){:+.3f}".format(A, B))
    
plot_model_annotate(X.view(-1).numpy(), Y.view(-1).numpy(),
                    linear_model.weight.item(), linear_model.bias.item(),
                    data.index)

Now we can train the model - minimise the loss function.

In [ ]:
def train_model(X, Y, model, loss_function, optim, num_epochs):
    loss_history = []
    
    def extra_plot(*args):
        plt.plot(X.numpy(), Y.numpy(), 'r.', label="Ground truth")
        plt.plot(X.numpy(), linear_model(X).detach().numpy(), '-', label="Model")
        plt.title("Prediction")
        plt.legend(loc='lower right')
    
    liveloss = PlotLosses(extra_plots=[extra_plot], plot_extrema=False)

    for epoch in range(num_epochs):
        
        epoch_loss = 0.0
        
        Y_pred = model(X)
        loss = loss_function(Y_pred, Y)
        
        loss.backward()
        optim.step()
        optim.zero_grad()
                
        epoch_loss = loss.data.item()
        
        avg_loss = epoch_loss / len(X)

        liveloss.update({
            'loss': avg_loss,
           #'a': model.weight[0][0].item(),
           #'b': model.bias[0].item()
        })
        liveloss.draw()
        print("y = {:.3f}x{:+.3f}".format(model.weight[0][0].item(), model.bias[0].item()))
        

train_model(X, Y, linear_model, loss_function, optim, num_epochs=50)

Let's see if we fitted the final line properly!

In [ ]:
plot_model_annotate(X.view(-1).numpy(), Y.view(-1).numpy(),
                    linear_model.weight.item(), linear_model.bias.item(),
                    data.index)

It fits the data much better than at the begining. We have found the relation between brain and body weights among various animal species:

In [ ]:
"log(PredictedBrainWeight) = {a:.3f} * log(BodyWeight) {b:.3f}".format(
    a=linear_model.weight[0][0].item(),
    b=linear_model.bias[0].item())

It can be transformed into an explicit formula:

In [ ]:
"PredictedBrainWeight =  {e_b:.3f} * (BodyWeight)^{e_a:.3f}".format(
    e_a=linear_model.weight[0][0].exp().item(),
    e_b=linear_model.bias[0].exp().item())

In [ ]:
# let's attach data
data['PredictedBrainWeight(kg)'] = linear_model(X).exp().detach().squeeze().numpy()
data['PredictedActualRatio'] = data['PredictedBrainWeight(kg)'] / data['BrainWeight(kg)']

We can now compare predicted brain weights with actual data. What does it mean that the actual brain weight is bigger than predicted one? Is an animal more clever in that case?

In [ ]:
data.sort_values(by='PredictedActualRatio').head(10)

Is it a good guess? Well, for me it sounds reasonable. At the same time, we don't have a simple way to compare intelligence of species operating in different environments, see [Cat vs. Squid by Wumo](http://wumo.com/wumo/2013/02/25):

![Cat vs. Squid | Wum](http://wumo.com/img/wumo/2013/02/25.png)

For food for though, crows are wicked smart, vide [Causal understanding of water displacement by a crow](https://www.youtube.com/watch?v=ZerUbHmuY04). [So are octopodes](https://www.nytimes.com/2018/11/30/science/animal-intelligence-octopus-cephalopods.html).

## Exercise

If you want to practice linear regression, here is another dataset. It describes the relation between weight and average heart rate of various animals. 

(Tip: try to scale the data, by taking logarithm of both values)

In [ ]:
heart_rate_dataset = pd.read_csv("data/Heart_rate_and_weight.csv", index_col=0)
heart_rate_dataset

## Extra

Here are some interesting websites on the subject of linear regression:


* [Linear regression](http://www.stat.yale.edu/Courses/1997-98/101/linreg.htm)
* [Ordinary Least Squares Regression-Explained Visually](http://setosa.io/ev/ordinary-least-squares-regression/)
* [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)


Beware, even if there is some correlation, it may be not that sound:

![https://imgs.xkcd.com/comics/linear_regression.png](https://imgs.xkcd.com/comics/linear_regression.png)

![https://imgs.xkcd.com/comics/extrapolating.png](https://imgs.xkcd.com/comics/extrapolating.png)

